### Notebook with Python code to generate data for MCEOF analysis of a multi-proxy network for paper: South American Monsoon variability over the last millennium in paleoclimate records and isotope-enabled climate models.

- Imports formatted age control and sampled d18O records
- Merges records to create standardized time series covering the period with annual resolution
- Performs Monte Carlo resampling to generate a proxy ensemble with 1,000 members
- EOF calculation and plotting (eof patterns and principal components)
- Saves output as CSV for EOF analysis and plotting applications

___

Rebecca Orrison 

In [ ]:
%reset

In [1]:
#----------
# system
#----------
import os

#----------
#additional packages
#----------
#data
from itertools import dropwhile
import math
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import xarray as xr

# computation
from eofs.standard import Eof
from eofs.xarray import Eof
import scipy
from scipy import interpolate
from scipy import signal
from scipy import ndimage
from scipy import stats
from sklearn import decomposition
from sklearn.decomposition import PCA
from statsmodels.multivariate.pca import PCA   # this is more of a climate perspective on PCA
from sklearn.preprocessing import StandardScaler


# plotting
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import cartopy.feature as cf
from matplotlib.gridspec import GridSpec
from matplotlib.image import imread
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

plt.rcParams.update({'figure.max_open_warning': 0})

/network/rit/lab/vuillelab_rit/orrison/bin/miniconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


# Define functions and classes

In [2]:
def lat_ind_fn(latl,latu):
    """Find the index of the latitude that corresponds to proxy latitude"""
    return np.where((lat >= latl) & (lat <= latu))
    
def lon_ind_fn(lonl,lonu):
    """Find the index of the longitude that corresponds to proxy longitude"""
    return np.where((lon >= lonl) & (lon <= lonu))

def nn_idx(target,array):
    return np.abs(array - target).argmin()

def GapFill(year,yng,old):
    slope = (records_dat[yng]['d18O'].iloc[-1] - records_dat[old]['d18O'][0]) / (xnew_ages[yng][j][[-1]][0] - xnew_ages[old][j][[0]][0])
    b = (xnew_ages[yng][j][[-1]][0]*records_dat[old]['d18O'][0] - xnew_ages[old][j][[0]][0]*records_dat[yng]['d18O'].iloc[-1])/(xnew_ages[yng][j][[-1]][0]-xnew_ages[old][j][[0]][0])
    return (slope * year) + b

def smonotonic(x):
    """Check if the array is strictly monotonic (incr or dec)"""
    dx = np.diff(x)
    return np.all(dx < 0) or np.all(dx > 0) 

def force_smonoton(x):
    """Forces an array that is not strictly monotonic to conform to strict monotonicity"""
    for i in range(len(x)-1):
        if x[i] >= x[i+1]:
            x[i+1] = x[i] + 0.01
    return x
#----------

class DataDict(dict):
    """ data_dict template """
    def __init__(self):         # create object
        self["record"] = []
        self["oxy_depth_mm"] = []
        self["year_CE"] = []
        self["'d18O'"] = []

### Load data

- age tie files
- isotope time series files

In [3]:
records = {}
pathi = "/network/rit/home/ro553136/orrison/data/proxy/mceof_recs/"
for file in os.listdir(pathi):
    if file.endswith('_ages.txt'):
        rec = os.path.splitext(file)[0].split('_')[0]
        records[rec] = pd.read_csv(os.path.join(pathi, file), sep='\t')
                
records_dat = {}
pathi = "/network/rit/home/ro553136/orrison/data/proxy/mceof_recs/"

for file in os.listdir(pathi):
    if file.endswith('_d18O.txt'):
        rec = os.path.splitext(file)[0].split('_')[0]
        records_dat[rec] = pd.read_csv(os.path.join(pathi, file), sep='\t')


# Sample compositing and Monte Carlo ensemble generation
- Monte Carlo resampling of age ties based on Gaussian uncertainty distribution
- Standard interpolation to annual resolution


In [94]:
#----------
# Define constants
#----------
size = 1000    # ensemble size
gauss_window = 30 
hlfwnd = gauss_window // 2
ages_comm = np.arange(850, 1850)    # last millennium time period
ages_comm = ages_comm[::-1]
lw_bnd = np.min(ages_comm) - hlfwnd
up_bnd = np.max(ages_comm) + hlfwnd

    # Final records names for EOF analysis
mceof_recs = ['PAL', 'HUA1', 'PAR', 'DV2', 'SBE+SMT', 'TMO', 'CRT1',
               'JAR', 'ALH', 'BOTO', 'PIM4', 'MV', 'QUELC3', 'PUM12']

#----------
# Pre-define arrays for data storage
#----------
age_mod_mc = {}
f_mc = {}
xnew = {}
x = {}
xnew_ages = {}
annages = {}
d18O_resamp = {}
d18O_comm = {}
d18O_comm_smth30 = {}
d18O_comm_smth10 = {}
rec_ages_list = []
y_list = []

#----------
# Framework for data analysis
#----------
for rec in records: 
        age_mod_mc[rec] = [[0.0 for i in range(len(records[rec]['tie_year_CE']))] for j in range(size)]
        xnew[rec] = np.linspace(np.min(records[rec]['tie_depth_mm']), np.max(records[rec]['tie_depth_mm']), np.max(records[rec]['tie_year_CE'])-np.min(records[rec]['tie_year_CE']))
        f_mc[rec] = [[0.0 for i in range(len(xnew[rec]))] for j in range(size)]
        x[rec] =  records[rec]['tie_depth_mm']
        xnew_ages[rec] = []
        d18O_resamp[rec] = [[None for i in range(len(xnew_ages[rec]))] for j in range(size)]    
        d18O_comm[rec] = [[None for i in range(len(ages_comm))] for j in range(size)] 
        d18O_comm_smth30[rec] = [[None for i in range(len(ages_comm))] for j in range(size)] 
        d18O_comm_smth10[rec] = [[None for i in range(len(ages_comm))] for j in range(size)] 

#----------
# Framework for records from merged samples and samples without an age model
#----------
newrecs = ['PUM12', 'SBE3' , 'QUELC3', 'JAR', 'PAR', 'SBE+SMT', 'PAL', 'BOTO', 'ALH', 'MV', 'PIM4']
for nrec in newrecs:
    xnew_ages.update({nrec : []})
    d18O_resamp.update({nrec : [[None for i in range(len(xnew_ages[rec]))] for j in range(size)] })
    d18O_comm.update({nrec : [[None for i in range(len(ages_comm))] for j in range(size)] })
    d18O_comm_smth10.update({nrec : [[None for i in range(up_bnd - lw_bnd)] for j in range(size)] })
    d18O_comm_smth30.update({nrec : [[None for i in range(up_bnd - lw_bnd)] for j in range(size)] })
    
#----------
# Framework for final records 
#----------  
for rec in mceof_recs:
       annages[rec] = []
#----------
# Age ties and d18O sample interpolation --> annually spaced 18O time series
#----------   
for j in range(size):
#     print('AGE MODEL RESAMPLING FOR ENS MEM ' + str(j))
    for rec in records:
        bol_flag = [False]*(len(records[rec]['tie_year_CE']))
            # resample age ties, make sure they don't violate superposition assumption
        for i in range(len(records[rec]['tie_year_CE'])):
    #             print('AGE TIE (#/total):')
    #             print(i, len(records[rec]['tie_year_CE']))
            while not bol_flag[i]:
                if i != len(records[rec]['tie_year_CE'])-1:
                    # generate new age tie resamples based on uncertainties in one of three ways: 
                    # check if second age tie violates superposition assump of the first age tie based on lower bounds
                    #if j is last age tie, don't do the below step.  indent the below.  
                    tie_min_diff = (records[rec]['tie_year_CE'][i]-records[rec]['err_2sig'][i]/2) - (records[rec]['tie_year_CE'][i+1]-records[rec]['err_2sig'][i+1]/2)
                    tie_max_diff = (records[rec]['tie_year_CE'][i]+records[rec]['err_2sig'][i]/2) - (records[rec]['tie_year_CE'][i+1]+records[rec]['err_2sig'][i+1]/2)
                    if tie_min_diff < 0:
                # generate truncated normal distribution of current age tie based on next one
                        lower = (records[rec]['tie_year_CE'][i+1]-records[rec]['err_2sig'][i+1]/2)+1
                        upper = records[rec]['tie_year_CE'][i]+records[rec]['err_2sig'][i]/2
                        mu = records[rec]['tie_year_CE'][i]
                        sigma = records[rec]['err_2sig'][i]/8
                        age_mod_mc[rec][j][i] = float(scipy.stats.truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=1))
                    elif tie_max_diff < 0:
                # generate truncated normal distribution of next age tie based on current one. 
                        lower = records[rec]['tie_year_CE'][i]-records[rec]['err_2sig'][i]/2
                        upper = (records[rec]['tie_year_CE'][i]+records[rec]['err_2sig'][i]/2)+1
                        mu = records[rec]['tie_year_CE'][i]
                        sigma = records[rec]['err_2sig'][i]/8
                        age_mod_mc[rec][j][i] = float(scipy.stats.truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=1))
                    else:
                        age_mod_mc[rec][j][i] = float(np.random.normal(loc=records[rec]['tie_year_CE'][i], scale=(records[rec]['err_2sig'][i]/8), size = 1))
                else:
                    age_mod_mc[rec][j][i] = float(np.random.normal(loc=records[rec]['tie_year_CE'][i], scale=(records[rec]['err_2sig'][i]/8), size = 1))
    #                 print()
    #                 print('age tie year +/- 1 sig uncertainty is')
    #                 print(records[rec]['tie_year_CE'][i], (records[rec]['err_2sig'][i]/2))
    #                 print('age tie resamp value is')
    #                 print(age_mod_mc[rec][j][i])
    #                 print('previous age tie value is')
    #                 print(age_mod_mc[rec][j][i-1])
    #                 print(np.shape(age_mod_mc[rec]))
    #                 print()
                if i == 0:
                    bol_flag[0] = True
    #                     print(bol_flag)
                elif i == len(records[rec]['tie_year_CE'])-1:
                    bol_flag[-1] = True
    #                     print(bol_flag)
                else: 
                    if (age_mod_mc[rec][j][i] - age_mod_mc[rec][j][i-1]) < -2: # verify sufficient separation of age ties
                        bol_flag[i] = True
    #                         print(bol_flag)
                    else:
                        print('resample age tie')
                        print(rec)

#                print('FINISHED AN AGE MODEL!')

        ### Generate function to calculate one age per isotopic sample depth
        ### new interpolated ages based on sampled depths and MC-derived age model
        f = interpolate.interp1d(records[rec]['tie_depth_mm'], age_mod_mc[rec][j][:], fill_value = "extrapolate")       
        xnew_ages[rec].append(np.array(f(records_dat[rec]['oxy_depth_mm'])))

#----------
# Clean up records, merge
#---------- 

#----- QUELC3
    xnew_ages['QUELC3'].append(records_dat['QUELC3']['year_CE'].values)
    d18O_resamp['QUELC3'][j][:] = records_dat['QUELC3']['d18O'].values 
    
#----- PUM12 
        ###  Annual records (exact yrs)
    pum12ann_yr = records_dat['PUM12an']['year_CE'].values
    pum12ann_dO = records_dat['PUM12an']['d18O'].values
    
        ###  Varved records, irregular sampling - interpolate to exact
    pum12vrv_yr = records_dat['PUM12v']['year_CE']
    pum12vrv_dO = records_dat['PUM12v']['d18O']
    pum12vrv_f = interpolate.interp1d(pum12vrv_yr,pum12vrv_dO, fill_value='extrapolate')
    pum12vrvann_yr = np.arange(840,1797) # input values for exact ann. interp of varved non-exact section
    pum12vrvann_yr = pum12vrvann_yr[::-1]
    pum12vrvann_dO = pum12vrv_f(pum12vrvann_yr)

        ### merge and reset PUM12 record 
    xnew_ages['PUM12'].append(np.concatenate([pum12ann_yr, pum12vrvann_yr]))
    d18O_resamp['PUM12'][j] = np.concatenate([pum12ann_dO, pum12vrvann_dO])
    
#----- ALH6
    alh6_irreg_yr = records_dat['ALH6']['year_CE']
    alh6_irreg_dO = records_dat['ALH6']['d18O']
    alh6_reg_f = interpolate.interp1d(alh6_irreg_yr, alh6_irreg_dO, fill_value='extrapolate')
    alh6_reg_d18O = alh6_reg_f(ages_comm)
    
        ### reset ALH6 record 
    xnew_ages['ALH'].append(ages_comm)
    d18O_resamp['ALH'][j] = alh6_reg_d18O
    
#---------- 
# Fill hiatus, merging samples to build composite records
#---------- 
#----- SBE3 a, b gap -> SBE3a + SBE3b = SBE3 
    gap_yrs = np.arange(xnew_ages['SBE3b'][j][0],xnew_ages['SBE3a'][j][-1])
    gap_yrs = gap_yrs[1:][::-1]
    gap_d18O = [GapFill(year,'SBE3a','SBE3b') for year in gap_yrs]
    ### Simple average of stdev 30 years before and after gap
    up_bnd = nn_idx(xnew_ages['SBE3a'][j][-1]+30, xnew_ages['SBE3a'][j])
    lw_bnd = nn_idx(xnew_ages['SBE3b'][j][0]-30, xnew_ages['SBE3b'][j])
    stda = np.std(records_dat['SBE3a']['d18O'][up_bnd:])
    stdb = np.std(records_dat['SBE3b']['d18O'][1:lw_bnd+1])
    std = (stda + stdb) / 2   

        ### resample and scale values within above std dev, shifting mean based on line.
    gap_d18O_resamp = np.concatenate([np.random.normal(loc=val,scale = std, size = 1) for val in gap_d18O])

        ### merge and reset SBE3 values
    xnew_ages['SBE3'].append(np.concatenate([xnew_ages['SBE3a'][j], gap_yrs, xnew_ages['SBE3b'][j]]))
    d18O_resamp['SBE3'][j] = np.concatenate([records_dat['SBE3a']['d18O'], gap_d18O_resamp, records_dat['SBE3b']['d18O']])

#----- PIM4 a, b gap -> PIM4a + PIM4b = PIM4   
    gap_yrs = np.arange(xnew_ages['PIM4b'][j][0],xnew_ages['PIM4a'][j][-1])
    gap_yrs_pim = gap_yrs[1:][::-1]
    gap_d18O = [GapFill(year,'PIM4a','PIM4b') for year in gap_yrs_pim]
    ### Simple average of stdev 30 years before and after gap
    up_bnd = nn_idx(xnew_ages['PIM4a'][j][-1]+30, xnew_ages['PIM4a'][j])
    lw_bnd = nn_idx(xnew_ages['PIM4b'][j][0]-30, xnew_ages['PIM4b'][j])
    stda = np.std(records_dat['PIM4a']['d18O'][up_bnd:])
    stdb = np.std(records_dat['PIM4b']['d18O'][1:lw_bnd+1])
    std = (stda + stdb) / 2   

        ### resample and scale values within above std dev, shifting mean based on line.
    gap_d18O_resamp = np.concatenate([np.random.normal(loc=val,scale = std, size = 1) for val in gap_d18O])

        ### merge and reset PIM4 values
    xnew_ages['PIM4'].append(np.concatenate([xnew_ages['PIM4a'][j], gap_yrs_pim, xnew_ages['PIM4b'][j]]))
    d18O_resamp['PIM4'][j] = np.concatenate([records_dat['PIM4a']['d18O'], gap_d18O_resamp, records_dat['PIM4b']['d18O']])

######     
#----- MV 
## Shift MV30, adjust xnew_ages for each sample fitting.
    MV30_correc = np.mean(records_dat['MV1']['d18O']) - np.mean(records_dat['MV30']['d18O'])
    records_dat['MV30']['d18O'] = records_dat['MV30']['d18O'] + MV30_correc
    gap_over = xnew_ages['MV30'][j][[0]]+1 - xnew_ages['MV1'][j][[-1]]

    ts1_d18 = np.array(records_dat['MV1']['d18O'])
    ts1_yr = np.array(xnew_ages['MV1'][j])
    ts2_d18 = np.array(records_dat['MV30']['d18O'])
    ts2_yr = np.array(xnew_ages['MV30'][j])

    if gap_over < 0: # gap to be filled. 

        strt1 = nn_idx(1850,ts1_yr)
        end2 = nn_idx(850,ts2_yr)
        ts1_zscr = (ts1_d18 - np.mean(ts1_d18[strt1::])) / np.std(ts1_d18[strt1::])
        ts2_zscr = (ts2_d18 - np.mean(ts2_d18[0:end2+1])) / np.std(ts2_d18[0:end2+1])    

        gap_yrs = np.arange(ts2_yr[0],ts1_yr[-1])
        gap_yrs_mv = gap_yrs[1:][::-1]
        slope = (ts2_zscr[0] - ts1_zscr[-1]) / (ts2_yr[[0]][0] - ts1_yr[[-1]][0])
        b = (ts2_yr[[0]][0]*ts1_zscr[-1] - ts1_yr[[-1]][0]*ts2_zscr[0])/(ts2_yr[[0]][0]-ts1_yr[[-1]][0])
        gap_d18O = [((slope * year) + b) for year in gap_yrs_mv]

        ### Simple average of stdev 30 years before and after gap
        up_bnd = nn_idx(xnew_ages['MV1'][j][-1]+30, xnew_ages['MV1'][j])
        lw_bnd = nn_idx(xnew_ages['MV30'][j][0]-30, xnew_ages['MV30'][j])
        stda = np.std(records_dat['MV1']['d18O'][up_bnd:])
        stdb = np.std(records_dat['MV30']['d18O'][1:lw_bnd+1])
        std = (stda + stdb) / 2    

            ### resample and scale values within above std dev, shifting mean based on line.
        gap_d18_resamp = np.concatenate([np.random.normal(loc=val,scale = std, size = 1) for val in gap_d18O])

            ### merge and reset MV values, scale to variance of MV1. 
        xnew_ages['MV'].append(np.concatenate([ts1_yr, gap_yrs_mv, ts2_yr]))
        d18_temp_comm = np.concatenate([ts1_zscr,gap_d18_resamp,ts2_zscr])
        d18O_resamp['MV'][j] = (d18_temp_comm * np.std(ts1_d18[strt1::])) + np.mean(ts1_d18[strt1::])

    elif gap_over > 2:  #  overlap exists: MERGE MV 1, 30

        ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

        p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
        p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

            # find overlap of stdardized ts
        ts1_d18_ovlp = ts1_d18[p2::]
        ts2_d18_ovlp = ts2_d18[0:p1+1]

           ### interpolate points with cubic spline. time must be monotonically increasing.
        f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
        f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

              ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
              ### Set ages young -> old; round to exact years
        sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
        sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
        sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

              ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
        ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
        ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 

              ### replace period of overlap with synchronized section for each time series
        ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
        ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
        ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:-1]])
        ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:-1]])

              ### standardize both records by mean, std of the records in the common period
        strt1 = nn_idx(1850,ts1_yr)
        end2 = nn_idx(850,ts2_yr)
        ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
        ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

            ### Add nan to buffer; ts?_zscr must have same length
        tmp = np.empty(len(ts2_yr[p1+1:]))
        tmp.fill(np.nan)
        ts1_zscr_full = np.concatenate((ts1_zscr, tmp))
        tmp = np.empty(len(ts1_yr[0:p2]))
        tmp.fill(np.nan)
        ts2_zscr_full = np.concatenate((tmp, ts2_zscr))

            ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
        mv_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)

        ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
        d18O_resamp['MV'][j] = (mv_zscr * np.std(ts1_d18O_full[strt1::])) + np.mean(ts1_d18O_full[strt1::])
        xnew_ages['MV'].append(np.concatenate([ts1_yr_full,ts2_yr[p1+1:-1]]))

    else: #years line up perfectly or have small enough gap that spline doesn't work:
            # still need to rescale MV30 to the std of MV1.
        strt1 = nn_idx(1850,ts1_yr)
        end2 = nn_idx(850,ts2_yr)
        ts1_zscr = (ts1_d18 - np.mean(ts1_d18[strt1::])) / np.std(ts1_d18[strt1::])
        ts2_zscr = (ts2_d18 - np.mean(ts2_d18[0:end2+1])) / np.std(ts2_d18[0:end2+1])

        xnew_ages['MV'].append(np.concatenate([ts1_yr,ts2_yr]))
        d18O_temp_comm = np.concatenate([ts1_zscr,ts2_zscr])

        d18O_resamp['MV'][j] = (d18O_temp_comm * np.std(ts1_d18[strt1::])) + np.mean(ts1_d18[strt1::])                                                 
                  
########     
#----- JAR 
    ts1_d18 = np.array(records_dat['JAR4']['d18O'])
    ts1_yr = np.array(xnew_ages['JAR4'][j])
    ts2_d18 = np.array(records_dat['JAR1']['d18O'])
    ts2_yr = np.array(xnew_ages['JAR1'][j])

    ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

    p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    ts1_d18_ovlp = ts1_d18[p2::]
    ts2_d18_ovlp = ts2_d18[0:p1+1]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
    f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
    sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
    ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
    ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
    ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:]])
    ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt1 = nn_idx(1850,ts1_yr_full)
    end2 = nn_idx(850,ts2_yr_full)
    ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
    ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmp = np.empty(len(ts2_yr[p1+1:]))
    tmp.fill(np.nan)
    ts1_zscr_full = np.concatenate((ts1_zscr, tmp))
    tmp = np.empty(len(ts1_yr[0:p2]))
    tmp.fill(np.nan)
    ts2_zscr_full = np.concatenate((tmp, ts2_zscr))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    jar_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)
    
    ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
    d18O_resamp['JAR'][j] = (jar_zscr * np.std(ts1_d18O_full[strt1::])) + np.mean(ts1_d18O_full[strt1::])
    xnew_ages['JAR'].append(np.concatenate([ts1_yr_full,ts2_yr[p1+1:]]))
                                    
########     
#----- 
# PAR
    ts1_d18 = np.array(records_dat['PAR03']['d18O'])
    ts1_yr = np.array(xnew_ages['PAR03'][j])
    ts2_d18 = np.array(records_dat['PAR01']['d18O'])
    ts2_yr = np.array(xnew_ages['PAR01'][j])

    ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

    p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    ts1_d18_ovlp = ts1_d18[p2::]
    ts2_d18_ovlp = ts2_d18[0:p1+1]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
    f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
    sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
    ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
    ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
    ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:]])
    ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt1 = nn_idx(1850,ts1_yr_full)
    end2 = nn_idx(850,ts2_yr_full)
    ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
    ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmp = np.empty(len(ts2_yr[p1+1:]))
    tmp.fill(np.nan)
    ts1_zscr_full = np.concatenate((ts1_zscr, tmp))
    tmp = np.empty(len(ts1_yr[0:p2]))
    tmp.fill(np.nan)
    ts2_zscr_full = np.concatenate((tmp, ts2_zscr))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    par_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)
    
    ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
    d18O_resamp['PAR'][j] = (par_zscr * np.std(ts1_d18O_full[strt1::])) + np.mean(ts1_d18O_full[strt1::])
    xnew_ages['PAR'].append(np.concatenate([ts1_yr_full,ts2_yr[p1+1:]]))
    
#----- 
# SBE+SMT
    ts1_d18 = np.array(d18O_resamp['SBE3'][j])
    ts1_yr = np.array(xnew_ages['SBE3'][j])
    ts2_d18 = np.array(records_dat['SMT5']['d18O'])
    ts2_yr = np.array(xnew_ages['SMT5'][j])

    ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

    p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    ts1_d18_ovlp = ts1_d18[p2::]
    ts2_d18_ovlp = ts2_d18[0:p1+1]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
    f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
    sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
    ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
    ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
    ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:]])
    ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt1 = nn_idx(1850,ts1_yr_full)
    end2 = nn_idx(850,ts2_yr_full)
    ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
    ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmp = np.empty(len(ts2_yr[p1+1:]))
    tmp.fill(np.nan)
    ts1_zscr_full = np.concatenate((ts1_zscr, tmp))
    tmp = np.empty(len(ts1_yr[0:p2]))
    tmp.fill(np.nan)
    ts2_zscr_full = np.concatenate((tmp, ts2_zscr))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    sbesmt_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)
    
    ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
    d18O_resamp['SBE+SMT'][j] = (sbesmt_zscr * np.std(ts1_d18O_full[strt1::])) + np.mean(ts1_d18O_full[strt1::])
    xnew_ages['SBE+SMT'].append(np.concatenate([ts1_yr_full,ts2_yr[p1+1:]]))
    
#----- 
# PAL3 and PAL4.  PAL 3 as a base. 
    ts1_d18 = np.array(records_dat['PAL03']['d18O'])
    ts1_yr = np.array(xnew_ages['PAL03'][j])
    ts2_d18 = np.array(records_dat['PAL04']['d18O'])
    ts2_yr = np.array(xnew_ages['PAL04'][j])

    ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

    p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    ts1_d18_ovlp = ts1_d18[p2::]
    ts2_d18_ovlp = ts2_d18[0:p1+1]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
    f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
    sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
    ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
    ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
    ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:]])
    ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt1 = nn_idx(1850,ts1_yr_full)
    end2 = nn_idx(850,ts2_yr_full)
    ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
    ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmp = np.empty(len(ts2_yr[p1+1:]))
    tmp.fill(np.nan)
    ts1_zscr_full = np.concatenate((ts1_zscr, tmp))
    tmp = np.empty(len(ts1_yr[0:p2]))
    tmp.fill(np.nan)
    ts2_zscr_full = np.concatenate((tmp, ts2_zscr))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    pal_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)
    
    ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
    d18O_resamp['PAL'][j] = (pal_zscr * np.std(ts1_d18O_full[strt1::])) + np.mean(ts1_d18O_full[strt1::])
    xnew_ages['PAL'].append(np.concatenate([ts1_yr_full,ts2_yr[p1+1:]]))

#----- 
# BOTO.  Boto 3 as base.  
 
    ts1_d18 = np.array(records_dat['BOTO7']['d18O'])
    ts1_yr = np.array(xnew_ages['BOTO7'][j])
    ts2_d18 = np.array(records_dat['BOTO3']['d18O'])
    ts2_yr = np.array(xnew_ages['BOTO3'][j])
    ts3_d18 = np.array(records_dat['BOTO10']['d18O'])
    ts3_yr = np.array(xnew_ages['BOTO10'][j])
    ts4_d18 = np.array(records_dat['BOTO1']['d18O'])
    ts4_yr = np.array(xnew_ages['BOTO1'][j])
    
    # MERGING BOTO3,BOTO7 into BOTOa

    ts1_d18O_cubresamp_ovlp, ts2_d18O_cubresamp_ovlp = [], []

    p1 = nn_idx(ts1_yr[-1],ts2_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p2 = nn_idx(ts2_yr[0],ts1_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    ts1_d18_ovlp = ts1_d18[p2::]
    ts2_d18_ovlp = ts2_d18[0:p1+1]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_ts1_ovlp_cubic = interpolate.CubicSpline(ts1_yr[p2::][::-1], ts1_d18_ovlp[::-1], bc_type='natural') 
    f_ts2_ovlp_cubic = interpolate.CubicSpline(ts2_yr[0:p1+1][::-1], ts2_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(ts1_yr[p2::][-1],ts2_yr[0:p1+1][-1]))
    sync_b = math.floor(min(ts1_yr[p2::][0],ts2_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    ts1_d18O_cubresamp_ovlp[:] = f_ts1_ovlp_cubic(sync_ages_ovlp) 
    ts2_d18O_cubresamp_ovlp[:] = f_ts2_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    ts1_d18O_full = np.concatenate([ts1_d18[0:p2], ts1_d18O_cubresamp_ovlp[::-1]])
    ts1_yr_full = np.concatenate([ts1_yr[0:p2],sync_ages_ovlp[::-1]])
    ts2_d18O_full = np.concatenate([ts2_d18O_cubresamp_ovlp[::-1], ts2_d18[p1+1:]])
    ts2_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts2_yr[p1+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt1 = nn_idx(1850,ts1_yr_full)
    end2 = nn_idx(850,ts2_yr_full)
    ts1_zscr = (ts1_d18O_full - np.mean(ts1_d18O_full[strt1::])) / np.std(ts1_d18O_full[strt1::])
    ts2_zscr = (ts2_d18O_full - np.mean(ts2_d18O_full[0:end2+1])) / np.std(ts2_d18O_full[0:end2+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmp1 = np.empty(len(ts2_yr[p1+1:]))
    tmp1.fill(np.nan)
    ts1_zscr_full = np.concatenate((ts1_zscr, tmp1))
    tmp2 = np.empty(len(ts1_yr[0:p2]))
    tmp2.fill(np.nan)
    ts2_zscr_full = np.concatenate((tmp2, ts2_zscr))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    botoa_zscr = np.nanmean( np.array([ts1_zscr_full,ts2_zscr_full]), axis = 0)
    
    botoa_d18_invrt = (botoa_zscr * np.std(ts2_d18O_full[0:end2+1])) + np.mean(ts2_d18O_full[0:end2+1])
    botoa_yr = np.concatenate([ts1_yr_full,ts2_yr[p1+1:]])
                                                                              
    # MERGING BOTOa,BOTO10

    tsa_d18O_cubresamp_ovlp, ts3_d18O_cubresamp_ovlp = [], []

    p3 = nn_idx(botoa_yr[-1],ts3_yr)   # index in ts1 for the end of overlap period.  search the time value of ts1 based on the start value of ts2
    p4 = nn_idx(ts3_yr[0],botoa_yr)   # index in ts2 for the start of overlap period.   search the time value of ts2 based on the end value of ts1. 

    botoa_d18_ovlp = botoa_d18_invrt[p4::]   
    ts3_d18_ovlp = ts3_d18[0:p3+1]     
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_tsa_ovlp_cubic = interpolate.CubicSpline(botoa_yr[p4::][::-1], botoa_d18_ovlp[::-1], bc_type='natural') 
    f_ts3_ovlp_cubic = interpolate.CubicSpline(ts3_yr[0:p3+1][::-1], ts3_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(botoa_yr[p4::][-1],ts3_yr[0:p3+1][-1]))
    sync_b = math.floor(min(botoa_yr[p4::][0],ts3_yr[0:p3+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    tsa_d18O_cubresamp_ovlp[:] = f_tsa_ovlp_cubic(sync_ages_ovlp) 
    ts3_d18O_cubresamp_ovlp[:] = f_ts3_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    tsa_d18O_full = np.concatenate([botoa_d18_invrt[0:p4], tsa_d18O_cubresamp_ovlp[::-1]])
    tsa_yr_full = np.concatenate([botoa_yr[0:p4],sync_ages_ovlp[::-1]])
    ts3_d18O_full = np.concatenate([ts3_d18O_cubresamp_ovlp[::-1],ts3_d18[p3+1:]])
    ts3_yr_full = np.concatenate([sync_ages_ovlp[::-1], ts3_yr[p3+1:]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt2 = nn_idx(1850,tsa_yr_full)
    end3 = nn_idx(850,ts3_yr_full)                                                                          
    tsa_zscr = (tsa_d18O_full - np.mean(tsa_d18O_full[strt2::])) / np.std(tsa_d18O_full[strt2::])
    ts3_zscr = (ts3_d18O_full - np.mean(ts3_d18O_full[0:end3+1])) / np.std(ts3_d18O_full[0:end3+1])

    ### Add nan to buffer; ts?_zscr must have same length
    tmpa = np.empty(len(ts3_yr[p3+1:]))
    tmpa.fill(np.nan)
    tsa_zscr_full = np.concatenate([tsa_zscr, tmpa])
    tmp3 = np.empty(len(botoa_yr[0:p4]))
    tmp3.fill(np.nan)
    ts3_zscr_full = np.concatenate([tmp3, ts3_zscr])

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    botob_zscr = np.nanmean( np.array([tsa_zscr_full,ts3_zscr_full]), axis = 0)
    
    botob_d18_invrt = (botob_zscr * np.std(tsa_d18O_full[strt1::])) + np.mean(tsa_d18O_full[strt1::])
    botob_yr = np.concatenate([tsa_yr_full,ts3_yr[p3+1:]])
        
    # MERGING BOTOb,BOTO1, Botob as base, boto1 embedded in it.
    
    tsb_d18O_cubresamp_ovlp, ts4_d18O_cubresamp_ovlp = [], []

    p5 = nn_idx(ts4_yr[-1],botob_yr)   # index in ts4 for the end of overlap period.  search the time value of botob based on the start value of ts2
    p6 = nn_idx(ts4_yr[0],botob_yr)   # index in ts4 for the start of overlap period.   search the time value of botob based on the end value of ts1. 

    tsb_d18_ovlp = botob_d18_invrt[p6:p5+1]
    ts4_d18_ovlp = ts4_d18[:]
   
        ### interpolate points with cubic spline. time must be monotonically increasing.
    f_tsb_ovlp_cubic = interpolate.CubicSpline(botob_yr[p6:p5+1][::-1], tsb_d18_ovlp[::-1], bc_type='natural') 
    f_ts4_ovlp_cubic = interpolate.CubicSpline(ts4_yr[::-1], ts4_d18_ovlp[::-1], bc_type='natural') 

          ### Synchronize: establish annual time series covered for each rec - use years from one series, matches closest points of the other series
          ### Set ages young -> old; round to exact years
    sync_a = math.ceil(max(botob_yr[p6:p5+1][-1],ts4_yr[0:p1+1][-1]))
    sync_b = math.floor(min(botob_yr[p6:p5+1][0],ts4_yr[0:p1+1][0]))
    sync_ages_ovlp = np.arange(sync_a, sync_b+1, 1.)  

#         ### Downscale cublic spline interpolation of d18O values to annual resolution in ovlp period 
    tsb_d18O_cubresamp_ovlp[:] = f_tsb_ovlp_cubic(sync_ages_ovlp) 
    ts4_d18O_cubresamp_ovlp[:] = f_ts4_ovlp_cubic(sync_ages_ovlp) 
    
    ### replace period of overlap with synchronized section for each time series
    tsb_d18O_full = np.concatenate([botob_d18_invrt[0:p6], tsb_d18O_cubresamp_ovlp[::-1],botob_d18_invrt[p5+1::]])
    tsb_yr_full = np.concatenate([botob_yr[0:p6],sync_ages_ovlp[::-1], botob_yr[p5+1::]])
    ts4_d18O_full = np.concatenate([ts4_d18O_cubresamp_ovlp[::-1]])
    ts4_yr_full = np.concatenate([sync_ages_ovlp[::-1]])

#     ### Standardize records (including replaced bit)) by mean, std of LM period
    strt4 = nn_idx(1850,tsb_yr_full)
    end5 = nn_idx(850,tsb_yr_full)
    tsb_zscr = (tsb_d18O_full - np.mean(tsb_d18O_full[strt4:end5+1])) / np.std(tsb_d18O_full[strt4:end5+1])
    ts4_zscr = (ts4_d18O_full - np.mean(ts4_d18O_full)) / np.std(ts4_d18O_full)

    ### Add nan to buffer; ts?_zscr must have same length
    tsb_zscr_full = tsb_zscr
    tmp4a = np.empty(len(botob_yr[0:p6]))
    tmp4a.fill(np.nan)
    tmp4b = np.empty(len(botob_yr[p5+1::]))
    tmp4b.fill(np.nan)
    ts4_zscr_full = np.concatenate((tmp4a, ts4_zscr, tmp4b))

    ### average two series. Use np.nanmean so that the period with record outside overlap with nans remains
    botoc_zscr = np.nanmean( np.array([tsb_zscr_full,ts4_zscr_full]), axis = 0)

    ### invert to LM mean, std values of record with longest sole LM coverage. Concatenate years together 
    d18O_resamp['BOTO'][j] = (botoc_zscr * np.std(tsb_d18O_full[strt4:end5+1])) + np.mean(tsb_d18O_full[strt4:end5+1])
    xnew_ages['BOTO'].append(tsb_yr_full)
    

/network/rit/lab/vuillelab_rit/orrison/bin/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


AGE MODEL RESAMPLING FOR ENS MEM 0
AGE MODEL RESAMPLING FOR ENS MEM 1
AGE MODEL RESAMPLING FOR ENS MEM 2


/network/rit/lab/vuillelab_rit/orrison/bin/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:413: RuntimeWarning: Mean of empty slice


AGE MODEL RESAMPLING FOR ENS MEM 3
resample age tie
PAL04
resample age tie
PAL04
AGE MODEL RESAMPLING FOR ENS MEM 4
AGE MODEL RESAMPLING FOR ENS MEM 5
AGE MODEL RESAMPLING FOR ENS MEM 6
AGE MODEL RESAMPLING FOR ENS MEM 7
resample age tie
PAL04
resample age tie
PAL04
resample age tie
PAL04
resample age tie
PAL04
resample age tie
PAL04
resample age tie
PAL04
AGE MODEL RESAMPLING FOR ENS MEM 8
AGE MODEL RESAMPLING FOR ENS MEM 9
AGE MODEL RESAMPLING FOR ENS MEM 10
AGE MODEL RESAMPLING FOR ENS MEM 11
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
resample age tie
BOTO10
AGE MODEL RESAMPLING FOR ENS MEM 12
AGE MODEL RESAMPLING FOR ENS MEM 13
AGE MODEL RESAMPLING FOR ENS MEM 14
AGE MODEL RESAMPLING FOR ENS MEM 15
resample age tie
PAL04
resample age tie
PAL04
AGE MODEL RESAMPLING 

In [95]:
#---------- 
# Generate new function for annual interpolation of d18O values. 
#----------     
for j in range(size):
    for rec in mceof_recs:
        if rec in ['HUA1', 'DV2', 'TMO', 'CRT1']:
            f_d18O_ann_linear = interpolate.interp1d(xnew_ages[rec][j], records_dat[rec]['d18O'], fill_value = "extrapolate") 
        else: 
            f_d18O_ann_linear = interpolate.interp1d(xnew_ages[rec][j], d18O_resamp[rec][j], fill_value = "extrapolate") 

        ### Establish annual time series covered for each rec, each ensemble member
        ### Set ages young -> old; round to exact years
        xnew_ages_tmp = np.arange(np.min(xnew_ages[rec][j]), np.max(xnew_ages[rec][j])+1, 1.)
        xnew_ages_tmp = np.around(xnew_ages_tmp[::-1])  
        annages[rec].append(xnew_ages_tmp)

        ### Linear interpolation of d18O values to annual resolution
        d18O_resamp[rec][j] = f_d18O_ann_linear(annages[rec][j]) 

    #---------- 
    # Truncate dataset to 850 -- 1850 CE
    #---------- 

        p1 = nn_idx(np.max(ages_comm),annages[rec][j])
        p2 = nn_idx(np.min(ages_comm),annages[rec][j])
        d18O_comm[rec][j] = d18O_resamp[rec][j][p1:p2+1]         

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Build data frame, save output of data as abs values and z scores

In [122]:
# Absolute vals: 
abs_prox_all = []
for iter in range(size):
    abs_prox = pd.DataFrame(d18O_comm[mceof_recs[0]][iter], index = ages_comm, columns=[mceof_recs[0]])
    for i in range(len(mceof_recs)-1):
        abs_prox.insert(i+1, value=(d18O_comm[mceof_recs[i+1]][iter]), column=mceof_recs[i+1])
  
    abs_prox_all.append(abs_prox)
    #print(iter)
    #print(abs_prox)
    
f = open("/network/rit/lab/vuillelab_rit/orrison/data/proxy/mceof_recs/mceof_recs_all_Sept21.csv","a")
for df in abs_prox_all:
    df.to_csv(f)
f.close()

In [123]:
# z scores: 
zmat_prox_all = []
for iter in range(size):
    zmat_prox = pd.DataFrame(stats.zscore(d18O_comm[mceof_recs[0]][iter]), index = ages_comm, columns=[mceof_recs[0]])
    for i in range(len(mceof_recs)-1):
        zmat_prox.insert(i+1, value=stats.zscore(d18O_comm[mceof_recs[i+1]][iter]), column=mceof_recs[i+1])
    
    zmat_prox_all.append(zmat_prox)
    #print(iter)
    #print(zmat_prox)

f = open("/network/rit/lab/vuillelab_rit/orrison/data/proxy/mceof_recs/mceof_zscores_all_Septv21.csv","a")
for df in zmat_prox_all:
    df.to_csv(f)
f.close()